In [67]:
import requests, re
from datetime import datetime
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup as bs

In [70]:
# Retrieve HTML data
url = "http://ke.customer.decent.cz/a022/mon/"
filename = "meu_data.xlsx"

# Scrape website

In [74]:
soup = bs(requests.get(url).content, 'html5lib')
tables = soup.find_all('table')

# # Get and format timestamp from first table.
# raw_datetime = tables[2].get_text()
# date = re.search(r'(\d+).(\d+).(\d+)', raw_datetime)
# time = re.search(r'(\d+)\s(\d+)', raw_datetime)

# # Create datetime object from data, not very pretty..
# timestamp = datetime.datetime(
#     int(date.group(3)),
#     int(date.group(2)),
#     int(date.group(1)), 
#     int(time.group(1)),
#     int(time.group(2)))

timestamp = datetime.now()

# Get data and import it into a Pandas DataFrame, format it.
raw_data = tables[4]
rows = raw_data.find_all('tr')
rows = [x.find_all('td') for x in rows]
data = [[col.get_text() for col in row] for row in rows]

headers = ['activity', 'queue', 'x','active_booths', 'wait_time','x']
table = pd.DataFrame( data[1:], columns=headers).set_index('activity')[:11]
table = table.drop('x', axis=1)

tables = {}

for col in table:
    tables[col] = pd.DataFrame(table[col].str.extract("(\d+)", expand=False).fillna('0'))
    tables[col].columns = [timestamp]
    tables[col] = tables[col].transpose()

# Create 3D array with queue, open booths, and wait time.
data_current = pd.Panel(tables)
data_current

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 1 (major_axis) x 11 (minor_axis)
Items axis: active_booths to wait_time
Major_axis axis: 2017-08-31 18:48:05.462643 to 2017-08-31 18:48:05.462643
Minor_axis axis: Žádost o cestovní pas to Evidence vozidel

# Open saved data

In [75]:
data_old = {}

# Check if file exists
if Path.is_file(Path(filename)):
    
    
    # Load excel, each sheet to separate dataframe and create Panel
    excel_data = pd.read_excel(filename, sheetname=None, header=0, index_col=0)
    excel_data = pd.Panel(excel_data)

    for sheet in excel_data:
        data_old[sheet] = excel_data[sheet].append(data_current[sheet])
    
    data_current = pd.Panel(data_old)

data_merged = data_current
data_merged

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 2 (major_axis) x 11 (minor_axis)
Items axis: active_booths to wait_time
Major_axis axis: 2017-08-31 07:11:00 to 2017-08-31 18:48:05.462643
Minor_axis axis: Žádost o cestovní pas to Evidence vozidel

# Write data to file

In [76]:
data_merged.to_excel(filename)